/tmp/ipykernel_3149/2645757369.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['xy'] = df[['start_x', 'start_y']]. apply(tuple, axis=1)
/tmp/ipykernel_3149/2645757369.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  end_df['xy'] = df[['end_x', 'end_y']]. apply(tuple, axis=1)
/tmp/ipykernel_3149/2645757369.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [10]:
hour_list = sorted(set(start_df.index.get_level_values(0)) | set(end_df.index.get_level_values(0)))

final_df = pd.DataFrame(columns=['time','grid_index','origin_count','destination_count'])

for hour in hour_list:
    dict = {}

    if hour in start_df.index:
        for row in start_df.loc[hour].itertuples():
            dict[row.Index] = {'origin_count': row.count, 'destination_count': 0}

    if hour in end_df.index:
        for row in end_df.loc[hour].itertuples():
            if row.Index in dict:
                dict[row.Index]['destination_count'] = row.count
            else:
                dict[row.Index] = {'origin_count': 0, 'destination_count': row.count}

    for grid_index, count in dict.items():
        final_df.loc[len(final_df)] = [hour, grid_index, count['origin_count'], count['destination_count']]

In [54]:
start_df.index.get_level_values(0) + end_df.index.get_level_values(0)

AttributeError: 'Index' object has no attribute 'list'

In [61]:
len(set(end_df.index.get_level_values(0)))

8304

In [60]:
len(sorted(set(start_df.index.get_level_values(0)) | set(end_df.index.get_level_values(0))))

8314

In [41]:
start_df.head(18)

count
start_time_utc xy             
2018-09-04T09  (57, 69)      2
2018-09-04T10  (54, 63)      1
               (57, 70)      1
               (59, 68)      1
               (62, 65)      1
2018-09-04T11  (51, 64)      2
               (55, 64)      2
               (57, 59)      1
               (57, 69)      2
               (57, 71)      1
               (58, 66)      1
               (59, 67)      1
               (59, 69)      5
               (62, 65)      1
               (62, 73)      1
               (63, 23)      1
               (63, 24)      1
               (64, 75)      1

In [24]:
df.drop(columns=['start_lat', 'start_lon', 'end_lat', 'end_lon'], inplace=True)

In [31]:
grid_max_y

125.73994949226454

In [29]:
df.describe()

,start_x,start_y,end_x,end_y
count,1.374665e+06,1.374665e+06,1.374665e+06,1.374665e+06
mean,5.861874e+01,6.707850e+01,5.861246e+01,6.704231e+01
std,4.195077e+00,8.751645e+00,4.485137e+00,8.858408e+00
min,5.046350e-01,2.833530e-01,7.242676e-01,1.331192e-02
25%,5.692373e+01,6.651180e+01,5.683737e+01,6.648087e+01
50%,5.851842e+01,6.889069e+01,5.852529e+01,6.888627e+01
75%,6.032928e+01,7.013863e+01,6.038322e+01,7.014245e+01
max,1.284121e+02,1.257196e+02,1.284536e+02,1.257196e+02


In [30]:
len(df)

1374665

In [21]:
df[df.isna().any(axis=1)]

,start_time_utc,end_time_utc,start_lat,start_lon,end_lat,end_lon


In [9]:
df['start_time_utc'] = df['start_time_utc'].str[: 13]

In [10]:
df.head()

,start_time_utc,end_time_utc,start_lat,start_lon,end_lat,end_lon
0,2018-09-04T09:,2018-09-04T10:27:00Z,39.766650,-86.160210,39.758495,-86.185028
1,2018-09-04T09:,2018-09-04T09:59:00Z,39.767559,-86.159926,39.768615,-86.158798
2,2018-09-04T10:,2018-09-04T10:15:00Z,39.777007,-86.146346,39.775085,-86.146225
3,2018-09-04T10:,2018-09-04T10:27:00Z,39.780555,-86.170685,39.778378,-86.178475
4,2018-09-04T10:,2018-09-04T10:50:00Z,39.769585,-86.156090,39.764035,-86.150330


In [2]:
utm.from_latlon(39.766650, -86.160210)

(571928.1083122492, 4402195.371753442, 16, 'S')

In [3]:
import pandas as pd
import ast
dataset = pd.read_csv('data/preprocessed_scooter_data.csv')

In [27]:
for name, group in dataset.groupby('time'):
    grid_index = group['grid_index'].apply(ast.literal_eval).tolist()
    grid_values = (group['destination_count'] - group['origin_count']).tolist()
    print(group)
    print(grid_values)
    print(grid_index)
    break


            time grid_index  origin_count  destination_count
0  2018-09-04T09   (57, 69)             2                  0
1  2018-09-04T09   (58, 68)             0                  1
[-2, 1]
[(57, 69), (58, 68)]


In [24]:
import torch

In [28]:
torch.sparse_coo_tensor(list(zip(*grid_index))[::-1], grid_values, (126, 130)).to_dense()[68:70, 57:59]

tensor([[ 0,  1],
        [-2,  0]])